| **Retrieval Technique**                    | **Description**                                                                                  | **Key Attributes**                                                                                               |
|--------------------------------------------|--------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------|
| **1. Vector Store Retrieval**              | Uses embeddings to find the most similar documents by comparing the vector representations.     | - `search_type="similarity"`: Finds similar documents. <br> - `k`: Number of documents to retrieve.            |
| **2. MultiQueryRetriever**                 | Generates multiple query variations to enhance document retrieval.                               | - Uses LLM to generate query variations. <br> - Helps expand query scope and improve retrieval.               |
| **3. MultiVectorRetriever**                | Stores multiple embeddings per document to improve search recall by considering different aspects. | - Uses multiple embeddings for each document. <br> - Improves recall by considering various document aspects.   |
| **4. Max Marginal Relevance (MMR) Retriever** | Ensures diversity among retrieved results by balancing similarity and diversity.                 | - `search_type="mmr"`: Ensures diversity. <br> - `lambda_mult`: Controls the balance between relevance and diversity. |
| **5. Self Query Retriever (Metadata Filtering)** | Filters documents using metadata, like publication year.                                          | - Uses metadata to filter documents. <br> - `metadata_field_info`: Defines the metadata fields for filtering.   |
| **6. BM25 (Lexical Search)**              | Uses traditional keyword-based search (term frequency, inverse document frequency).             | - Uses keyword matching. <br> - No embeddings involved.                                                        |
| **7. Hybrid Search (BM25 + Vector Search)** | Combines keyword-based search (BM25) and embedding-based search for more robust results.        | - `EnsembleRetriever`: Combines multiple retrievers. <br> - Weights for each retriever.                          |


In [18]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.retrievers import BM25Retriever, MultiQueryRetriever, MultiVectorRetriever, SelfQueryRetriever
from langchain.retrievers import EnsembleRetriever

# Step 1: Load webpage content and filter for relevant parts (title, headers, content).
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
docs = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
).load()

# Step 2: Split the content into manageable chunks.
recursive_text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=200, separators=["\n\n", "\n", " ", ""])
page_content = [doc.page_content for doc in docs]
chunks = []
for content in page_content:
    chunks.extend(recursive_text_splitter.split_text(content))

# Step 3: Generate embeddings using Ollama model.
embed = OllamaEmbeddings(model="nomic-embed-text")
embeddings = embed.embed_documents(chunks)

# Step 4: Create and store the FAISS vector store with the embeddings.
vectorstore = FAISS.from_embeddings(list(zip(chunks, embeddings)), embedding=embed)

In [28]:
#Groq LLM
import os
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = "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"
llm= ChatGroq(model="mixtral-8x7b-32768")

In [29]:
# --- Retrieval Techniques ---

# 1. **Vector Store Retrieval (Similarity Search)**  
retriever_similarity = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs_similarity = retriever_similarity.invoke("What are the approaches to Task Decomposition?")
print("Similarity Search:", len(retrieved_docs_similarity))

Similarity Search: 6


In [30]:
# 1. **Similarity Score Threshold Retrieval**  
retriever_threshold = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.8})
retrieved_docs_threshold = retriever_threshold.invoke("What are the approaches to Task Decomposition?")
print("Similarity Score Threshold:", len(retrieved_docs_threshold))

No relevant docs were retrieved using the relevance score threshold 0.8


Similarity Score Threshold: 0


In [31]:
# 2. **MultiQueryRetriever (Query Variations)**  
multi_query_retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm=llm)
retrieved_docs_multi_query = multi_query_retriever.invoke("What are the approaches to Task Decomposition?")
print("MultiQueryRetriever:", len(retrieved_docs_multi_query))

MultiQueryRetriever: 5


In [32]:
# 3. **MultiVectorRetriever (Multiple Embeddings per Document)**  
from langchain_core.stores import InMemoryByteStore

multi_vector_retriever = MultiVectorRetriever(vectorstore=vectorstore, byte_store=InMemoryByteStore())
retrieved_docs_multi_vector = multi_vector_retriever.invoke("What are the approaches to Task Decomposition?")
print("MultiVectorRetriever:", len(retrieved_docs_multi_vector))

MultiVectorRetriever: 0


In [33]:
# 4. **Max Marginal Relevance (MMR)**  
retriever_mmr = vectorstore.as_retriever(search_type="mmr", search_kwargs={'k': 6, 'lambda_mult': 0.25})
retrieved_docs_mmr = retriever_mmr.invoke("What are the approaches to Task Decomposition?")
print("Max Marginal Relevance (MMR):", len(retrieved_docs_mmr))

Max Marginal Relevance (MMR): 6


In [ ]:
# 5. **Self Query Retriever (Metadata Filtering)**  
metadata_field_info = {"date": "2023-06-23", "author": "Lilian Weng"}  # Example metadata info
self_query_retriever = SelfQueryRetriever.from_llm(vectorstore, llm, metadata_field_info=metadata_field_info,document_contents=page_content)
retrieved_docs_self_query = self_query_retriever.invoke("What are the approaches to Task Decomposition published on 2023-06-23?")
print("Self Query Retriever:", len(retrieved_docs_self_query))

In [44]:
# 6. **BM25 (Lexical Search)**  
from langchain.schema import Document

documents = [Document(page_content=content) for content in page_content]
bm25_retriever = BM25Retriever.from_documents(documents)  # Using chunks instead of full documents
retrieved_docs_bm25 = bm25_retriever.invoke("What are the approaches to Task Decomposition?")
print("BM25 Retrieval:", len(retrieved_docs_bm25))

BM25 Retrieval: 1


In [45]:
# 7. **Hybrid Search (BM25 + Vector Search)**  
hybrid_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever_similarity], weights=[0.5, 0.5])
retrieved_docs_hybrid = hybrid_retriever.invoke("What are the approaches to Task Decomposition?")
print("Hybrid Search (BM25 + Vector):", len(retrieved_docs_hybrid))

Hybrid Search (BM25 + Vector): 7
